<a href="https://colab.research.google.com/github/WRFitch/fyp/blob/main/src/fyp_preliminary_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup


*   Install & import necessary libraries
*   Set up Earth Engine datastores. 



In [1]:
!pip install fastai
!pip install fastai2
!pip install tensorflow

     |████████████████████████████████| 184kB 8.4MB/s 
     |████████████████████████████████| 61kB 6.6MB/s 


In [2]:
import ee
import folium
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from fastai.tabular import *
from IPython.display import Image

In [4]:
ee.Authenticate()
ee.Initialize()

#print(fastai.__version__)
print(folium.__version__)
print(tf.__version__)

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=bllH2dDDbXdCP11gNOvySbG3p_enfrDpTosOuUE3HSA&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g7dSLJcMf0fSkMsuQ1Flq-I-yVpvCqtEpW_CdAIw338vZd72rjUvPc

Successfully saved authorization token.
0.8.3
2.4.0



### Import the following datasets into google drive beforehand:

*   [Sentinel-2 Satellite photography](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR)
*   [Sentinel-5 Precursor Data](https://developers.google.com/earth-engine/datasets/catalog/sentinel)
  *   [Aerosol](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_AER_AI)
  *   [Cloud](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_CLOUD)
  *   [Carbon Monoxide](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_CO)
  *   [Formaldehyde](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_HCHO)
  *   [Nitrogen Dioxide](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_NO2)
  *   [Ozone](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_O3)
  *   [Sulphur Dioxide](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_SO2)
  *   [Methane](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_CH4)
*   [ODIAC Fossil Fuel CO2 Emissions](https://db.cger.nies.go.jp/dataset/ODIAC/DL_odiac2019.html)


In [45]:
# Earth engine username, used to import classified image into ee assets folder
USERNAME = 'wrfitch'
OUTPUT_DIR = USERNAME + "/out/"

# define image collections for each dataset to be used 
s2 = ee.ImageCollection("COPERNICUS/S2_SR")
s5_CO = ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_CO")
s5_HCHO = ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_HCHO") 
s5_NO2 = ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_NO2")
s5_O3 = ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_O3")
s5_SO2 = ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_SO2")
s5_CH4 = ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_CH4")

# other variables that only need to be defined once. 
start_date = '2020-01-01'
end_date = '2020-12-31'
vis_palette = ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']

In [28]:
# define utility methods - TODO remove to utils script 

# pre-filter to remove clouds - we can add them back in as data points from sentinel... 5? 
def maskS2clouds(image) :
  qa = image.select('QA60');

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
      .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

  return image.updateMask(mask).divide(10000)

In [46]:
# Import datasets 
# TODO analyse whether these min/max values are valid, recalibrate for highest variance where necessary
# TODO analyse whether it makes sense to analyse these on a highly localised level

# High-resolution satellite photograph 
s2img = ee.ImageCollection('COPERNICUS/S2_SR') \
                  .filterDate(start_date, end_date) \
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
                  .map(maskS2clouds).median()
s2_id = s2img.getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3})

# Carbon monoxide
# Minmax scale is a bit off - recalibrate for Britain
CO_img = s5_CO.filterDate(start_date, end_date) \
              .select('CO_column_number_density').mean()
CO_id = CO_img.getMapId( \
    {'palette': vis_palette, \
    'min': 0, \
    'max': 0.05})

# Formaldehyde
# Minmax scale is a bit off - recalibrate for Britain
HCHO_img = s5_HCHO.filterDate(start_date, end_date) \
                  .select('tropospheric_HCHO_column_number_density').mean()
HCHO_id = HCHO_img.getMapId( \
    {'palette': vis_palette, \
    'min': 0.0, \
    'max': 0.0003})

# Nitrogen Dioxide
NO2_img = s5_NO2.filterDate(start_date, end_date) \
                .select('tropospheric_NO2_column_number_density').mean()
NO2_id = NO2_img.getMapId( \
    {'palette': vis_palette, \
    'min': 0.0, \
    'max': 0.0002})

# Ozone
O3_img = s5_O3.filterDate(start_date, end_date) \
              .select('O3_column_number_density').mean()
O3_id = O3_img.getMapId( \
    {'palette': vis_palette, \
    'min': 0.12, \
    'max': 0.15})

# Sulphur Dioxide
SO2_img = s5_SO2.filterDate(start_date, end_date) \
              .select('SO2_column_number_density').mean()
SO2_id = SO2_img.getMapId( \
    {'palette': vis_palette, \
    'min': 0.0, \
    'max': 0.0005})

# Methane
CH4_img = s5_CH4.filterDate(start_date, end_date) \
                .select('CH4_column_volume_mixing_ratio_dry_air').mean()
CH4_id = CH4_img.getMapId( \
    {'palette': vis_palette, \
    'min': 1750, \
    'max': 1900})


In [48]:
# Visualise data on a Folium map 
map = folium.Map(location=[51.5, 0.1])

folium.TileLayer(
    tiles=s2_id['tile_fetcher'].url_format,
    #TODO update attr field
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='median composite',
  ).add_to(map)

folium.TileLayer(
    tiles=CO_id['tile_fetcher'].url_format,
    #TODO update attr field
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Carbon Monoxide',
  ).add_to(map)

folium.TileLayer(
    tiles=HCHO_id['tile_fetcher'].url_format,
    #TODO update attr field
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Formaldehyde',
  ).add_to(map)

folium.TileLayer(
    tiles=NO2_id['tile_fetcher'].url_format,
    #TODO update attr field
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Nitrogen Dioxide',
  ).add_to(map)

folium.TileLayer(
    tiles=O3_id['tile_fetcher'].url_format,
    #TODO update attr field
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Ozone',
  ).add_to(map)

folium.TileLayer(
    tiles=SO2_id['tile_fetcher'].url_format,
    #TODO update attr field
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Sulphur Dioxide',
  ).add_to(map)

folium.TileLayer(
    tiles=CH4_id['tile_fetcher'].url_format,
    #TODO update attr field
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Methane',
  ).add_to(map)

map.add_child(folium.LayerControl())
map

In [ ]:
from fastai.vision import *

path = untar_data(URLs.PETS)/'images'

In [ ]:

type(get_image_files(path))